In [21]:
import json
import requests
import numpy as np
import pandas as pd
import matplotlib as plt
import sqlite3
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

In [22]:
url = 'https://api.yelp.com/v3/businesses/search'
url2 = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [23]:
client_id = "0RHbihxg1H8-FtXWdWRtfQ"
api_key = "7Y9_AJxCRZODCh-ltNRyXKnetWfEbh9IukEMhXM1bGMeNpgl3Xpr1f5nwAIuj2wLKNilp770vvggIP1fW1uFc26-YolKbvCjf6b8GrWLee-fY3yaxwU3uUGdW0nzX3Yx"

In [24]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

In [25]:
term = 'yoga'
location = 'Manhattan'
radius = 10000

In [26]:
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "radius" : radius,
                "limit": 50
            }

In [27]:
def yelp_call(url, url_params, api_key):
    # your code to make the yelp call
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    return data

In [28]:
NYCdata = yelp_call(url, url_params, api_key)

In [29]:
cur.execute("""CREATE TABLE businesses (
                                id INTEGER PIRMARY KEY, 
                                businessID TEXT,
                                rating INTEGER,
                                price TEXT,
                                name TEXT,
                                review_count TEXT,
                                city TEXT,
                                country TEXT
                                )          
            """)
conn.commit()

In [30]:
cur.execute("""CREATE TABLE reviews (
                                businessID TEXT,
                                reveiewID TEXT,
                                rating INTEGER,
                                time_created TEXT,
                                text TEXT
                                )          
            """)
conn.commit()

In [13]:
def parse_results(results):
    biz_list = []
    biz_list_id = []

    for business in results['businesses']:
        for item in ['businessID', 'rating', 'price', 'name', 'review_count', 'city', 'country']:
            if item not in business:
                business[item] = np.nan
            else: 
                business[item]
        biz_tuple = ( business['id'], 
                      business['rating'],
                      business['price'],
                      business['name'],
                      business['review_count'],
                      business['location']['city'],
                      business['location']['country']
                      )    
        biz_list_id.append(business['id'])
        biz_list.append(biz_tuple)
    return biz_list

In [14]:
def insert_values(conn, cur, parse_results):
    for business in parse_results:
        add_business = ("""
    INSERT INTO businesses (businessID, rating, price, name, review_count, city, country)
    VALUES (?, ?, ?, ?, ?, ?, ?) """)
        cur.executemany(add_business, parse_results)
        conn.commit()
        

In [15]:
# create a variable  to keep track of which result you are in. 
current_offset = 0
#num = NYCdata['total']
#set up a while loop to go through and grab the result 
while current_offset < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = current_offset
    #make your API call with the new offset number
    results = yelp_call(url, url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    insert_values(conn, cur, parsed_results)
    #increment the counter by 50 to move on to the next results
    current_offset += 50
    print(current_offset)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000


In [16]:
ID_list = []

for i in range(0, len(NYCdata['businesses'])):
    for k,v in NYCdata['businesses'][i].items():
        if k == 'id':
            ID_list.append(v)

ID_list

['kmj7FnZsKPPH3psbR9tX7Q',
 'GgvXfu8U80tcrxythjpB4w',
 'TlDcvf1cEK2im6e4AFYe3A',
 'Oi85AYz9-Bx-9SBjNf__vg',
 'npgq_up0_puDV5jieUza-A',
 'nuhFcWqgHBwmgwlbm4rI9w',
 'Chuu09nqfEQbREmaphoOuw',
 'FK_6tqTQuSkadiWQ1w8ttQ',
 'XpY0Cxo_Iw2deqeY6zsNOA',
 'nkUwnPPYmCICZ8nYCu_twg',
 'tGtrhu0YDGkM9ECrYVz6xg',
 'omwcHHgdgUJn-W1b3i78og',
 'Mc17IR6PG4iZLmZzciJEQQ',
 'oECxp7RoWT2QArWgqpwnMA',
 'x0bjd5AUu3ASzfjmht2IYA',
 'c_80f3N3NbWDX8RnvuM3Xg',
 '89VGtFw9HZASM0IdnSEeFA',
 'KjSI9xkCFTrRRbHOzCG7xg',
 'PZhmOvUx0HrdGvxVrxTJ5g',
 'b1ziC3xpS5Bb9PGsufBYAw',
 'NqL3lVZgh1pz0tVaZBxkRg',
 'vxNBDgBMP44E8DHuqj5z6A',
 'I3-EsRgXDcLx3OrlACCx3Q',
 '6-fZD_ZzKJo29zapElbOZg',
 'dzvqCMHOsvruyOZZ8wSnbA',
 'HcsOCtxnSpAmUU0EC_AGMQ',
 'qFJcpI5TOBQkX6BukZqUVw',
 'PISWOLCiOj47QbZKm-F3tg',
 'cwADiq0xORmv4QA2jhu-Pw',
 'JvwVi-ZOfGKQEiezZTCgEA',
 'fiLGdKwhkvK6pZ3E_GjHmQ',
 'qhjcMZivcYfr7CaTHkiXXg',
 'P5OZCF6xChJtWNrjNFBBRA',
 'hW_eQNgyPkfwAH4Gq6I6LQ',
 'gbRr9zRxpBtBLum9kMlxZw',
 'Vi_qAjnkRk4kCaMpx_-Gkg',
 'Vwy58_3fyh1LwcMCFGb5vQ',
 

In [17]:
def yelp_call_reviews(lists):
    for ID in lists:
        url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(ID)
    # your code to make the yelp call
    response = requests.get(url, headers=headers)
    data = response.json()
    return data

In [20]:
yelp_call_reviews(ID_list)

{'reviews': [{'id': '65UsNFbTkqKPT9ezUI3PmA',
   'url': 'https://www.yelp.com/biz/dharma-yoga-center-new-york-2?adjust_creative=0RHbihxg1H8-FtXWdWRtfQ&hrid=65UsNFbTkqKPT9ezUI3PmA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=0RHbihxg1H8-FtXWdWRtfQ',
   'text': 'Hatha-Raja yoga at its best. Clean and spacious, this beautiful studio reminded me of a temple. I discovered this studio years ago when one of my kundalini...',
   'rating': 5,
   'time_created': '2020-02-02 05:50:17',
   'user': {'id': 'fd1HH94cxSMd7kQowYcP3A',
    'profile_url': 'https://www.yelp.com/user_details?userid=fd1HH94cxSMd7kQowYcP3A',
    'image_url': 'https://s3-media3.fl.yelpcdn.com/photo/MkVKGXpKx6DygtLxn0w7bA/o.jpg',
    'name': 'Cynthia Z.'}},
  {'id': '61woNfU1OtVLfDilARHYVg',
   'url': 'https://www.yelp.com/biz/dharma-yoga-center-new-york-2?adjust_creative=0RHbihxg1H8-FtXWdWRtfQ&hrid=61woNfU1OtVLfDilARHYVg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=0RHbihxg1